June: Using Michael's modules to calculate the coordinates and scaling factors from the raw grid.

In [1]:
import scipy.io as sio
import numpy as np
import netCDF4 as nc
import sys
sys.path.append('/ocean/mdunphy/MEOPAR/analysis-michael/bathymetry/')
import coordinates_helpers

The functions in [coordinates_helpers.py](https://bitbucket.org/salishsea/analysis-michael/src/03a3976ec3d8c70868b384bd9bdc4bc408444f37/bathymetry/coordinates_helpers.py?at=default&fileviewer=file-view-default) are read into [coordinates_redo.py](https://bitbucket.org/salishsea/analysis-michael/src/03a3976ec3d8c70868b384bd9bdc4bc408444f37/bathymetry/coordinates_redo.py?at=default&fileviewer=file-view-default).

In [2]:
help(coordinates_helpers)

Help on module coordinates_helpers:

NAME
    coordinates_helpers

FUNCTIONS
    expandi(lon, lat)
    
    expandj(lon, lat)
    
    gete1(lon, lat, expandleft=False)
    
    gete2(lon, lat, expanddown=False)
    
    haversine(lon1, lat1, lon2, lat2)
        This is copied from salishsea_tools
    
    t2f(lont, latt)
    
    t2u(lont, latt)
    
    t2v(lont, latt)
    
    writecoords(fname, glamt, glamu, glamv, glamf, gphit, gphiu, gphiv, gphif, e1t, e1u, e1v, e1f, e2t, e2u, e2v, e2f)

FILE
    /ocean/mdunphy/MEOPAR/analysis-michael/bathymetry/coordinates_helpers.py




Here I am looking at how the coordinates file was created for the Salish Sea.

```python
mfile = sio.loadmat('/ocean/mdunphy/MEOPAR/WORK/Coordinates-201702/seagrid_west_coast_1km_900x400_rot_new.mat')

michael_lons = mfile['s'][0,0]['geographic_grids'][0,0]
michael_lats = mfile['s'][0,0]['geographic_grids'][0,1]

print(michael_lons, michael_lats)

(array([[-123.42943248, -123.43197046, -123.43464085, ..., -126.40379914,
         -126.40732828, -126.41091045],
        [-123.4241149 , -123.42677215, -123.42948009, ..., -126.39801553,
         -126.40145667, -126.40484899],
        [-123.41879733, -123.42152316, -123.42427182, ..., -126.39224616,
         -126.39563389, -126.39897786],
        ..., 
        [-121.31303761, -121.31641017, -121.31978272, ..., -124.34068745,
         -124.34393562, -124.34713086],
        [-121.30772003, -121.31109299, -121.31446594, ..., -124.33616652,
         -124.33945967, -124.34269022],
        [-121.30240246, -121.30577581, -121.30914916, ..., -124.33167296,
         -124.33504631, -124.33841967]]),
 array([[ 46.85966577,  46.86278736,  46.86607161, ...,  50.39395285,
          50.39799966,  50.40210692],
        [ 46.86154594,  46.86481399,  46.86814418, ...,  50.39604719,
          50.40011363,  50.40422505],
        [ 46.86342605,  46.86677822,  46.87015824, ...,  50.39811478,
          50.4021822 ,  50.40627658],
        ..., 
        [ 47.60277441,  47.60681867,  47.61086262, ...,  51.11017828,
          51.11398796,  51.11781555],
        [ 47.6046284 ,  47.60867287,  47.61271704, ...,  51.1117346 ,
          51.11552856,  51.11934375],
        [ 47.60648232,  47.61052701,  47.6145714 , ...,  51.11328145,
          51.11704756,  51.12081337]]))
          

print(michael_lons.shape, michael_lats.shape)

(401, 901) (401, 901)

michael_glamt = np.transpose( michael_lons )
michael_gphit = np.transpose( michael_lats )

michael_glamt.shape, michael_gphit.shape
((901, 401), (901, 401))

```

glamt and gphit (longitude and latitude) are in the shape (y, x) after being transposed. 

The code below is Michael's [coordinates_redo.py](https://bitbucket.org/salishsea/analysis-michael/src/03a3976ec3d8c70868b384bd9bdc4bc408444f37/bathymetry/coordinates_redo.py?at=default&fileviewer=file-view-default). My geographic grids are stored in an .nc file instead a .mat file. Also, my grids are in (x,y) format, so I will keep the transpose. I will also change the output slices since those numbers are specifically for the Salish Sea.

But, first, I'll check the Mackenzie grid.

In [3]:
grid_file = nc.Dataset('/ocean/imachuca/Canyons/mackenzie_canyon/coordinates/raw_coordinates/grid_quad.nc')

mackenzie_lons = grid_file['grid_lons']
mackenzie_lats = grid_file['grid_lats']

mackenzie_lons.shape

(483, 363)

In [4]:
glamt = np.transpose( mackenzie_lons )
gphit = np.transpose( mackenzie_lats )

glamt.shape, gphit.shape

((363, 483), (363, 483))

In [5]:
from IPython import embed
from coordinates_helpers import *
import scipy.io as sio
import netCDF4 as nc
import numpy as np
import datetime

# This script rebuilds the coordinates file for SalishSea based on the
# original seagrid output found in a .mat file in the NEMO_PREPARATION archive.
#
# We use the same grid points as in the original coordinates file, but
# recompute the scaling factors because they were not correct in the
# original file.
#
# For reference, the Arakawa-C grid looks like this:
#           f---v---f---v---f  
#           |       |       |
#           u   T   u   T   u
#           |       |       |
#           f---v---f---v---f
#

# Load JP's original seagrid output, which are our T points
# After transposing, the dimensions are 901x401 while SalishSea is 898x398.
# We drop the last three in both dimensions when writing which gives us the
# same grid as in the original coordinates file. However we keep the extra
# points during the computations so we can get scaling factors at the right/top.

#mfile = sio.loadmat('seagrid_west_coast_1km_900x400_rot_new.mat')
#glamt = np.transpose( mfile['s'][0,0]['geographic_grids'][0,0] )
#gphit = np.transpose( mfile['s'][0,0]['geographic_grids'][0,1] )

# Compute the rest of the grid points
glamu,gphiu = t2u(glamt,gphit)
glamv,gphiv = t2v(glamt,gphit)
glamf,gphif = t2f(glamt,gphit)

# Compute scaling factors (with extrapolation for the left/bottom most scaling factor)
#
e1t = gete1(glamu,gphiu,expandleft=True)   # Need a left u point
e1u = gete1(glamt,gphit)
e1v = gete1(glamf,gphif,expandleft=True)   # Need a left f point
e1f = gete1(glamv,gphiv)
#
e2t = gete2(glamv,gphiv,expanddown=True)   # Need a lower v point
e2u = gete2(glamf,gphif,expanddown=True)   # Need a lower f point
e2v = gete2(glamt,gphit)
e2f = gete2(glamu,gphiu)

I will also change this output slice number to y-3 and x-3 using the shape of glamt. This will give me a final output of (y-3, x-3) which matches the bathymetry shapes.

In [6]:
output_slice_y = glamt.shape[0] - 3
output_slice_x = glamt.shape[1] - 3

output_slice_y, output_slice_x

(360, 480)

In [7]:
# Output slices
J,I = slice(0,output_slice_y), slice(0,output_slice_x)

#
filename = "../../coordinates/NEMO_files/coords_quad.nc"
writecoords(filename,
            glamt[J,I],glamu[J,I],glamv[J,I],glamf[J,I],
            gphit[J,I],gphiu[J,I],gphiv[J,I],gphif[J,I],
            e1t[J,I],e1u[J,I],e1v[J,I],e1f[J,I],
            e2t[J,I],e2u[J,I],e2v[J,I],e2f[J,I])

# Add note to history
cnc = nc.Dataset(filename, 'r+')
note ='[{}] Rebuilt with correct scaling factors (e1* and e2*)'
cnc.setncattr('history', note.format(datetime.datetime.today().strftime('%Y-%m-%d')))
cnc.close()

In [9]:
! ls ../../coordinates/NEMO_files/

coords_01.nc  coords_02.nc  coords_quad.nc


In [12]:
new_coords_file = nc.Dataset('../../coordinates/NEMO_files/coords_quad.nc')
new_coords_file

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    history: [2017-06-28] Rebuilt with correct scaling factors (e1* and e2*)
    dimensions(sizes): x(480), y(360), time(1)
    variables(dimensions): float32 nav_lon(y,x), float32 nav_lat(y,x), float32 time(time), float64 glamt(time,y,x), float64 glamu(time,y,x), float64 glamv(time,y,x), float64 glamf(time,y,x), float64 gphit(time,y,x), float64 gphiu(time,y,x), float64 gphiv(time,y,x), float64 gphif(time,y,x), float64 e1t(time,y,x), float64 e1u(time,y,x), float64 e1v(time,y,x), float64 e1f(time,y,x), float64 e2t(time,y,x), float64 e2u(time,y,x), float64 e2v(time,y,x), float64 e2f(time,y,x)
    groups: 